## Parte 3 - Classificação - Algoritmos

---
<img align="center" width="150" src="https://logodownload.org/wp-content/uploads/2017/05/ifood-logo-0.png">



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
import numpy as np
import pandas as pd

## Lendos os Dados

In [2]:
df_abt_train = pd.read_csv('/content/abt_classificacao_ifood_train.csv')
df_abt_train.head(2)

,data_ref,customer_id,uf,receita_1m,qtd_pedidos_1m,recencia,churn_next_month
0,2019-07-01 00:00:00+00:00,0001a8e61d8b08ad436e8e6f4adeb399b88df962c72d9d...,PR,102.0,2,15,1
1,2019-07-01 00:00:00+00:00,0001a9f97d01d2696cf70c7657ee2d039388d691720ff9...,SP,52.0,1,19,1


In [3]:
df_abt_oot = pd.read_csv('/content/abt_classificacao_ifood_oot.csv')
df_abt_oot.head(2)

,data_ref,customer_id,uf,receita_1m,qtd_pedidos_1m,recencia,churn_next_month
0,2019-08-01 00:00:00+00:00,000a1fac4f7a67cc3f2e7667167597cb2c9a1b9edafe18...,SP,85.89,3,11,0
1,2019-08-01 00:00:00+00:00,0014b7013c66a05d0b5ce0687d614ac220d3ae1af398d2...,PE,211.96,5,11,0


In [4]:
key_vars = ['data_ref', 'customer_id']
num_vars = ['receita_1m', 'qtd_pedidos_1m', 'recencia']
cat_vars = ['uf']
target = 'churn_next_month'
features = cat_vars + num_vars

In [5]:
# Dados de treinamento somente com as variáveis numéricas
X_train = df_abt_train[features]
y_train = df_abt_train[target]

# Dados de avaliação (out of time) somente com as variáveis numéricas
X_oot = df_abt_oot[features]
y_oot = df_abt_oot[target]

In [6]:
print(X_train.shape, y_train.shape)

(30079, 4) (30079,)


In [7]:
print(X_oot.shape, y_oot.shape)

(18441, 4) (18441,)


**`Q1`** Treine uma Random Forest na base de treino usando Stratified K-Fold e avalie utilizando a métrica **AUC** na base Out of Time.

- Utilize o parametro `random_state=42` no algoritmo.
- Utilize os seguintes parâmetros `n_splits=5, shuffle=True, random_state=42` para Stratified K-Fold.
- Lembre-se de realizar o pré-processamento dos dados

In [22]:
pip install scikit-learn==1.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 93.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [8]:
pip install feature-engine

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import MeanMedianImputer, CategoricalImputer
from feature_engine.encoding import OneHotEncoder

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
#pré-processamento com Pipeline + treino
rf_pipe = Pipeline(
    steps = [
    ("numeric_imputer", MeanMedianImputer(variables=num_vars, imputation_method='median')),
    ("categorical_imputer", CategoricalImputer(variables = cat_vars, fill_value = "missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("rf",RandomForestClassifier(random_state=42))
]
)

In [12]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, cross_validate

In [13]:
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [14]:
cv_rf = cross_validate(rf_pipe, X_train, y_train, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"], cv=skf, n_jobs=-1, error_score='raise')

In [15]:
cv_rf = pd.DataFrame(cv_rf)
cv_rf.mean()

,0
fit_time,6.441133
score_time,0.795856
test_accuracy,0.637189
test_precision,0.533140
test_recall,0.500689
test_f1,0.516361
test_roc_auc,0.687022


In [16]:
from sklearn.metrics import accuracy_score, precision_score,f1_score,recall_score,roc_auc_score

In [18]:
rf_pipe.fit(X_train, y_train)

Pipeline(steps=[('numeric_imputer',
                 MeanMedianImputer(variables=['receita_1m', 'qtd_pedidos_1m',
                                              'recencia'])),
                ('categorical_imputer',
                 CategoricalImputer(fill_value='missing', variables=['uf'])),
                ('ohe', OneHotEncoder(variables=['uf'])),
                ('rf', RandomForestClassifier(random_state=42))])

In [20]:
predicoes_proba_rf = rf_pipe.predict_proba(X_oot)[:,1]
roc_rf = roc_auc_score(y_oot,predicoes_proba_rf)
roc_rf

0.6804526243869495

**`Q2`** Treine um LGBM na base de treino usando Stratified K-Fold e avalie utilizando a métrica **AUC** na base Out of Time.

- Utilize o parametro `random_state=42` no algoritmo.
- Utilize os seguintes parâmetros `n_splits=5, shuffle=True, random_state=42` para Stratified K-Fold.
- Lembre-se de realizar o pré-processamento dos dados

In [21]:
pip install lightgbm

In [22]:
from lightgbm import LGBMClassifier

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [23]:
lg_pipe = Pipeline(
    steps = [
    ("numeric_imputer", MeanMedianImputer(variables=num_vars, imputation_method='median')), #se vierem faltando numerico
    ("categorical_imputer", CategoricalImputer(variables = cat_vars, fill_value = "missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("lg",LGBMClassifier(random_state=42))
]
)

In [27]:
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
cv_lg = cross_validate(lg_pipe, X_train, y_train, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"], cv=skf, n_jobs=-1, error_score='raise')
cv_lg = pd.DataFrame(cv_lg)
cv_lg.mean()

,0
fit_time,0.792117
score_time,0.320618
test_accuracy,0.691346
test_precision,0.594300
test_recall,0.637311
test_f1,0.615015
test_roc_auc,0.752186


In [28]:
lg_pipe.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 11638, number of negative: 18441
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001634 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 30079, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.386914 -> initscore=-0.460301
[LightGBM] [Info] Start training from score -0.460301


Pipeline(steps=[('numeric_imputer',
                 MeanMedianImputer(variables=['receita_1m', 'qtd_pedidos_1m',
                                              'recencia'])),
                ('categorical_imputer',
                 CategoricalImputer(fill_value='missing', variables=['uf'])),
                ('ohe', OneHotEncoder(variables=['uf'])),
                ('lg', LGBMClassifier(random_state=42))])

In [29]:
predicoes_proba_lg = lg_pipe.predict_proba(X_oot)[:,1]
roc_lg = roc_auc_score(y_oot,predicoes_proba_lg)
roc_lg

0.7332745686567761

**`Q3`** Treine um CatBoost na base de treino usando Stratified K-Fold e avalie utilizando a métrica **AUC** na base Out of Time.



In [30]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.4 MB/s eta 0:00:00


In [31]:
from catboost import CatBoostClassifier

In [32]:
cat_pipe = Pipeline(
    steps = [
    ("numeric_imputer", MeanMedianImputer(variables=num_vars, imputation_method='median')), #se vierem faltando numerico
    ("categorical_imputer", CategoricalImputer(variables = cat_vars, fill_value = "missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("cat",CatBoostClassifier(random_state=42))
]
)

In [33]:
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
cv_cat = cross_validate(cat_pipe, X_train, y_train, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"], cv=skf, n_jobs=-1, error_score='raise')
cv_cat = pd.DataFrame(cv_cat)
cv_cat.mean()

,0
fit_time,17.088502
score_time,0.402237
test_accuracy,0.691014
test_precision,0.595904
test_recall,0.625711
test_f1,0.610411
test_roc_auc,0.752546


In [34]:
cat_pipe.fit(X_train, y_train)

Learning rate set to 0.044071
0:	learn: 0.6805587	total: 71.2ms	remaining: 1m 11s
1:	learn: 0.6694912	total: 90.6ms	remaining: 45.2s
2:	learn: 0.6595210	total: 111ms	remaining: 36.8s
3:	learn: 0.6501366	total: 128ms	remaining: 31.9s
4:	learn: 0.6419883	total: 149ms	remaining: 29.7s
5:	learn: 0.6345792	total: 168ms	remaining: 27.9s
6:	learn: 0.6279115	total: 185ms	remaining: 26.2s
7:	learn: 0.6216539	total: 197ms	remaining: 24.5s
8:	learn: 0.6164232	total: 214ms	remaining: 23.5s
9:	learn: 0.6113737	total: 231ms	remaining: 22.9s
10:	learn: 0.6066090	total: 249ms	remaining: 22.4s
11:	learn: 0.6028462	total: 267ms	remaining: 22s
12:	learn: 0.5988335	total: 280ms	remaining: 21.3s
13:	learn: 0.5952078	total: 295ms	remaining: 20.8s
14:	learn: 0.5920015	total: 313ms	remaining: 20.6s
15:	learn: 0.5890553	total: 332ms	remaining: 20.4s
16:	learn: 0.5865849	total: 353ms	remaining: 20.4s
17:	learn: 0.5843112	total: 373ms	remaining: 20.3s
18:	learn: 0.5821240	total: 389ms	remaining: 20.1s
19:	learn:

Pipeline(steps=[('numeric_imputer',
                 MeanMedianImputer(variables=['receita_1m', 'qtd_pedidos_1m',
                                              'recencia'])),
                ('categorical_imputer',
                 CategoricalImputer(fill_value='missing', variables=['uf'])),
                ('ohe', OneHotEncoder(variables=['uf'])),
                ('cat',
                 <catboost.core.CatBoostClassifier object at 0x7dd4cf88f5d0>)])

In [35]:
predicoes_proba_cat = cat_pipe.predict_proba(X_oot)[:,1]
roc_cat = roc_auc_score(y_oot,predicoes_proba_cat)
roc_cat

0.7335190176031258